In [1]:
!pip install pyevall

In [2]:
!git clone https://github.com/dierop/ALCLabs

fatal: destination path 'ALCLabs' already exists and is not an empty directory.


In [7]:
cd ALCLabs/Lab2/


/content/ALCLabs/Lab2


In [18]:
!git checkout update
!git pull

Already on 'update'
Your branch is up to date with 'origin/update'.
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 13 (delta 8), reused 13 (delta 8), pack-reused 0 (from 0)
Unpacking objects: 100% (13/13), 5.13 KiB | 875.00 KiB/s, done.
From https://github.com/dierop/ALCLabs
   2aab2c3..10e06f5  update     -> origin/update
Updating 2aab2c3..10e06f5
Fast-forward
 Lab2/bert.py          |  6 ++++--
 Lab2/classic.py       | 11 +++++++----
 Lab2/outputs/rf.json  |  1 +
 Lab2/outputs/svc.json |  1 +
 Lab2/resultados.py    | 23 +++++++++++++----------
 Lab2/runs.ipynb       | 63 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++--
 6 files changed, 87 insertions(+), 18 deletions(-)
 create mode 100644 Lab2/outputs/rf.json
 create mode 100644 Lab2/outputs/svc.json


In [13]:
!ls

bert.py     data	   evaluate.py	  __pycache__	  resultados.py
classic.py  dataloader.py  preprocess.py  pyproject.toml  runs.ipynb


In [21]:
from preprocess import preprocess_data
from classic import train_and_evaluate
from sklearn.model_selection import train_test_split
import pandas as pd
from bert import train_and_evaluate_bert
from evaluate import store_results
from dataloader import load_data_json, load_test_json, load_data_csv, load_blip_csv


# Datos
path = "data/" # add Lab2/data if run from outisde Lab2
train_data_path = f"{path}lab2_materials/dataset_task4_exist2025/training.json"
train_data_path_blip = f'{path}lab2_materials/dataset_task4_exist2025/blip_captions_training.csv'
train_data_path_mami = f'{path}mami_dataset/training_mami.csv'

test_data_path = f"{path}lab2_materials/dataset_task4_exist2025/test.json"
test_data_path_blip = f'{path}lab2_materials/dataset_task4_exist2025/blip_captions_test.csv'
test_data_path_mami = f'{path}mami_dataset/test_mami.csv'

def main(blip=0, mami=False, gen_test=False):
    print("Blip: ", blip)
    print("Mami: ", mami)
    print("Gen Test: ", gen_test)
    # Data Loader
    train_data_meme = load_data_json(train_data_path)
    train_data_blip = load_blip_csv(train_data_path_blip)
    train_data_mami = load_data_csv(train_data_path_mami)

    test_data_blip = load_blip_csv(test_data_path_blip)
    test_data_mami = load_data_csv(test_data_path_mami)

    # Conjunto a evaluar
    test_data = load_test_json(test_data_path)

    if blip == 1:
        train_data_meme['text'] = train_data_blip['text']
        test_data['text'] = test_data_blip['text']
    elif blip == 2:
        train_data_meme['text'] = train_data_meme['text'].astype(str)
        train_data_blip['text'] = train_data_blip['text'].astype(str)
        train_data_meme['text'] = train_data_meme['text'].str.cat(train_data_blip['text'], sep=' ')
        test_data['text'] = test_data['text'].str.cat(test_data_blip['text'], sep=' ')



    # Preprocesar datos
    train_data_meme['text'] = train_data_meme['text'].apply(preprocess_data)
    #train_data_blip['blip_caption'] = train_data_blip['blip_caption'].apply(preprocess_data)
    train_data_mami['text'] = train_data_mami['text'].apply(preprocess_data)

    test_data['text'] = test_data['text'].apply(preprocess_data)
    #test_data_blip['blip_caption'] = test_data_blip['blip_caption'].apply(preprocess_data)
    test_data_mami['text'] = test_data_mami['text'].apply(preprocess_data)


    # Consolidado
    X = train_data_meme['text']
    y = train_data_meme['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Add MAMI data to train set
    # It is used as data augmentation
    if mami:
        for data in [train_data_mami, test_data_mami]:
            X = data['text']
            y = data['label']
            X_train = pd.concat([X_train, X], ignore_index=True)
            y_train = pd.concat([y_train, y], ignore_index=True)

    if not gen_test:
    # Resultados SVC
        # accuracy_svc, f1_svc, y_preds = train_and_evaluate(X_train, y_train, X_test, y_test, kernel="linear", tpye="SVC")
        # print("SVC")
        # print(accuracy_svc, f1_svc)

        # # Resultados RF
        # accuracy_rf, f1_rf, y_preds = train_and_evaluate(X_train, y_train, X_test, y_test, kernel="linear", tpye="RF")
        # print("RF")
        # print(accuracy_rf, f1_rf)

        # Resultados BERT
        accuracy_bert, f1_bert, y_preds = train_and_evaluate_bert(X_train.tolist()[:5000], y_train.tolist()[:5000], X_test.tolist(), y_test.tolist(), epochs=3, batch_size=16)
        print(accuracy_bert, f1_bert)
    else:
        X_train = pd.concat([X_train, X_test], ignore_index=True)
        y_train = pd.concat([y_train, y_test], ignore_index=True)
        # Resultados BERT
        y_preds = train_and_evaluate_bert(X_train.tolist()[:5000], y_train.tolist()[:5000], test_data['text'].tolist(), [0] * len(test_data['text']))
        store_results(test_data['id'], y_preds, "bert", path="outputs")

        # # Resultados SVC
        # y_preds = train_and_evaluate(X_train, y_train, test_data['text'], [], kernel="linear", tpye="SVC")
        # store_results(test_data['id'], y_preds, "svc", path="outputs")

        # # Resultados RF
        # _,_,y_preds = train_and_evaluate(X_train, y_train, test_data['text'], [], kernel="linear", tpye="RF")
        # store_results(test_data['id'], y_preds, "rf", path="outputs")

In [15]:

for b in range(3):

    main(b, False)


Blip:  0
Mami:  False
Gen Test:  False


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1/3, Loss: 0.6805
Epoch 2/3, Loss: 0.6367
Epoch 3/3, Loss: 0.5808
0.6532 0.6569
Blip:  1
Mami:  False
Gen Test:  False
Epoch 1/3, Loss: 0.6828
Epoch 2/3, Loss: 0.6759
Epoch 3/3, Loss: 0.6562
0.6055 0.5831
Blip:  2
Mami:  False
Gen Test:  False
Epoch 1/3, Loss: 0.6703
Epoch 2/3, Loss: 0.6044
Epoch 3/3, Loss: 0.4975
0.6917 0.6862


In [16]:
main(2, True)

Blip:  2
Mami:  True
Gen Test:  False
Epoch 1/3, Loss: 0.6334
Epoch 2/3, Loss: 0.5658
Epoch 3/3, Loss: 0.4782
0.6752 0.6791


In [17]:
main(0, True)

Blip:  0
Mami:  True
Gen Test:  False
Epoch 1/3, Loss: 0.6510
Epoch 2/3, Loss: 0.5803
Epoch 3/3, Loss: 0.5101
0.6716 0.665


In [22]:
main(2, False, True)

Blip:  2
Mami:  False
Gen Test:  True
Epoch 1/3, Loss: 0.6770
Epoch 2/3, Loss: 0.6163
Epoch 3/3, Loss: 0.5372


In [25]:
cd outputs/


/content/ALCLabs/Lab2/outputs


In [27]:
dowload bert.json

SyntaxError: invalid syntax (<ipython-input-27-6de5e4f0ad7f>, line 1)